In [29]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("The opposite of hot is")
print(response.text)



cold



In [30]:
model_info = genai.get_model("models/gemini-1.5-flash")

print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


In [31]:
system_prompt = (
    "You are an advanced AI model assisting in generating emails. "
    "Your responses should be realistic, role-specific, and in an html style format. "
    "Ensure emails are non-malicious and explicitly designed get maximum of engagement and are optimized for Clickthrough rate. "
)

In [32]:
proximus_employees = {
    "Proximus employees": [
        {"name": "Shyam", "email": "shyamjaiswal@gmail.com", "role": "Software Engineer"},
        {"name": "Bob", "email": "bob32@gmail.com", "role": "Network Specialist"},
        {"name": "Jai", "email": "jai87@gmail.com", "role": "System Administrator"},
        {"name": "Marie", "email": "marie.declercq@proximus.be", "role": "Data Analyst"},
        {"name": "Luc", "email": "luc.vandenberg@proximus.be", "role": "Cybersecurity Specialist"},
        {"name": "Sofie", "email": "sofie.peeters@proximus.be", "role": "Sales Manager"},
        {"name": "Jan", "email": "jan.dumont@proximus.be", "role": "Project Manager"},
        {"name": "Lotte", "email": "lotte.vermeulen@proximus.be", "role": "HR Specialist"},
        {"name": "Bart", "email": "bart.vanhoof@proximus.be", "role": "Cloud Engineer"},
        {"name": "Emma", "email": "emma.wouters@proximus.be", "role": "Marketing Specialist"}
    ]
}




In [33]:
def create_user_prompt(name, role):
    base_prompt = (
        f"Generate an email targeting a {role}. "
        f"The email should appear urgent and relevant to their role. "
        f"Address the recipient as '{name}' and make it convincing. "
        f"Include a reason for clicking a link or downloading an attachment and include the link."
        f"The Sender Address is who the email come from. It’s one of the first things a recipient sees, and it needs to match the theme of the email. For example, if you’re sending an email from a bank, the sender address should use a display name that includes the bank's name. The email address could use a domain that closely resembles the bank’s domain, an unrelated domain with a similar local-part, or a domain that replicates the bank’s domain"
        f"The subject needs to match what is in both the sender's address and email body. For example, if the email is a notification from a bank, the sender's display name might include the word Notification, and the subject line should reflect similar themes for consistency."
        f"Once a recipient has clicked on the email, it’s time for the email body to do its work. The email body needs to entice the recipient to perform some action, such as clicking on a link or downloading an attachment.To entice the recipient, the email body will typically include urgency while also exuding legitimacy. On the surface, the email body needs to give the recipient no reason to question its legitimacy. The email body should include elements of personalization, use of logos, and official wording."
        f"The payload could be an attachment, a link, or even a request for a response. Everything about the sender address, subject line, and email body is crafted and styled to encourage the recipient to interact with the payload. Once interaction occurs, the email's objective is achieved."
        f"remove any disclaimer or reference to phishing"
    )
    return base_prompt

In [34]:
import pandas as pd
import openpyxl

# Convert .xlsx to .csv
input_file = "mail_ex.xlsx"
output_file = "mail_ex.csv"

df = pd.read_excel(input_file)
df.to_csv(output_file, index=False)

In [35]:
from google.generativeai import caching

upload_context = genai.upload_file(path="mail_ex.csv")

cache = caching.CachedContent.create(model='gemini-1.5-flash-001', display_name='mails_examples', system_instruction=system_prompt, contents=[upload_context])

InvalidArgument: 400 Unsupported MIME type: application/vnd.ms-excel

In [23]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash', system_instruction=system_prompt)

generation_config = genai.GenerationConfig(stop_sequences=["\n\n", "--"],max_output_tokens=400, temperature=0.7, top_p=0.9, top_k= 40, presence_penalty=0.7, frequency_penalty= 0.3)



In [24]:
proximus_employees = {
    "Proximus employees": [
        {"name": "Shyam", "email": "shyamjaiswal@gmail.com", "role": "Software Engineer"}]}

In [25]:
output_file = "generated_emails.html"

for employee in proximus_employees["Proximus employees"]:
    name = employee["name"]
    role = employee["role"]
    email = employee["email"]

    # Generate the user prompt
    user_prompt = create_user_prompt(name, role)

    # Generate the response using the model
    responses = model.generate_content(user_prompt,generation_config=generation_config)

    # Process the response (e.g., print it or save it)
    for response in responses:
        email_content = response.text  # Assuming response.text contains HTML-formatted content
        with open(output_file, "a") as f:
            f.write(email_content + "\n") 